In [93]:
import pywbemReq

c = pywbemReq.WBEMConnection('https://10.245.48.46:5989', ('nasadmin', 'nasadmin'), 'root/emc/celerra')



In [96]:
##### Create NFS Filesystem #####

fs_service = c.EnumerateInstances('Celerra_FileSystemConfigurationService')[0]
print('##### Using FS service:')
print(fs_service.tomof())

nfs_server = c.EnumerateInstances('Celerra_NFSServer')[0]
print('##### Using NFS server:')
print(nfs_server.tomof())

nfs_capabilities = c.Associators(nfs_server.path, ResultClass='SNIA_LocallyAccessibleFileSystemCapabilities')[0]
print('##### Using Filesystem capabilities:')
print(nfs_capabilities.tomof())

result = c.InvokeMethod('CreateGoalSettings', nfs_capabilities.path)
local_access_setting = result[1]['SupportedGoalSettings'][0]
print('##### Using Local access settings:')
print(local_access_setting.tomof())

pool = c.EnumerateInstances('Celerra_NonPrimordialStoragePool')[0]
print('##### Using Pool:')
print(pool.tomof())

result = c.InvokeMethod('SNIA_CreateFileSystem', fs_service.path,
                        Sizes=[pywbemReq.Uint64(10737418240)],
                        FileServer=nfs_server.path, ElementName='fs_nfs_liangr',
                        Pools=[pool.path],
                        LocalAccessSetting=local_access_setting
                       )
print('##### Creating filesystem...')
print('##### Result:')
print(result)

##### Using FS service:
instance of Celerra_FileSystemConfigurationService {
	ElementName = "APM001432104072007";
	Name = "EMC-NSG:root/emc/celerra:APM001432104072007:FSCS";
	Caption = "FSCS:APM001432104072007";
	SystemName = "APM001432104072007";
	CreationClassName = "Celerra_FileSystemConfigurationService";
	SystemCreationClassName = "Celerra_CelerraComputerSystem";
};

##### Using NFS server:
instance of Celerra_NFSServer {
	Dedicated = {16};
	Name = "APM001432104072007:server_2:NFSServer";
	ElementName = "NFSS:APM001432104072007:server_2:NFSServer";
	NameFormat = "Other";
	Caption = "NFSS:APM001432104072007:server_2:NFSServer";
	OperationalStatus = {2};
	CreationClassName = "Celerra_NFSServer";
	OtherIdentifyingInfo = {"APM001432104072007:server_2:NFSServer"};
};

##### Using Filesystem capabilities:
instance of Celerra_LocallyAccessibleFileSystemCapabilities {
	InstanceID = "EMC-NSG:root/emc/celerra:APM001432104072007:LAFSC:server_2:NFSServer";
	Caption = "LAFSC:server_2:NFSServer

In [99]:
##### Create NFS Share #####

nfs_server = c.EnumerateInstances('Celerra_NFSServer')[0]
print('##### Using NFS server:')
print(nfs_server.tomof())

nfs_share_service = c.Associators(nfs_server.path, ResultClass= 'CIM_FileExportService')[0]
print('##### Using NFS share service:')
print(nfs_share_service.tomof())

nfs_share_capabilities = c.Associators(nfs_share_service.path,
                                        AssocClass='CIM_ElementCapabilities',
                                        ResultClass='CIM_ExportedFileShareCapabilities')[0]
print('##### Using NFS expport capabilities:')
print(nfs_share_capabilities.tomof())

default_no_access, default_ro, default_rw = pywbemReq.Uint16(2), pywbemReq.Uint16(3), pywbemReq.Uint16(4)

nfs_share_name = 'nfs_liangr'
fs_name = 'fs_nfs_liangr'

fs = filter(lambda x: x['ElementName'] == fs_name, c.EnumerateInstances('CIM_FileSystem'))[0]
print('##### Using File System:')
print(fs.tomof())

result = c.InvokeMethod('CreateGoalSettings', nfs_share_capabilities.path)
nfs_goalsettings = result[1]['SupportedGoalSettings'][0]
nfs_goalsettings['DefaultReadWrite'] = default_rw
print('##### Using NFS Goal Setting:')
print(nfs_goalsettings.tomof())


result = c.InvokeMethod('SNIA_CreateExportedShare',
                        nfs_share_service.path,
                        ElementName=nfs_share_name,
                        Root=fs.path,
                        #SharedElementPath='/',
                        Goal=nfs_goalsettings)
print('##### Creating NFS share ...')
print('##### Result:')
print(result)
nfs_share = c.GetInstance(result[1]['TheShare'])
print('##### NFS share created:')
print(nfs_share.tomof())

##### Using NFS server:
instance of Celerra_NFSServer {
	Dedicated = {16};
	Name = "APM001432104072007:server_2:NFSServer";
	ElementName = "NFSS:APM001432104072007:server_2:NFSServer";
	NameFormat = "Other";
	Caption = "NFSS:APM001432104072007:server_2:NFSServer";
	OperationalStatus = {2};
	CreationClassName = "Celerra_NFSServer";
	OtherIdentifyingInfo = {"APM001432104072007:server_2:NFSServer"};
};

##### Using NFS share service:
instance of Celerra_FileExportService {
	ElementName = "server_2:NFSServer";
	Name = "server_2:NFSServer";
	Caption = "FES:server_2:NFSServer";
	SystemName = "server_2";
	CreationClassName = "Celerra_FileExportService";
	SystemCreationClassName = "Celerra_NFSServer";
};

##### Using NFS expport capabilities:
instance of Celerra_ExportedFileShareCapabilities {
	CASupported = False;
	ElementName = "NFSServer";
	InstanceID = "EMC-NSG:root/emc/celerra:server_2:EFSC:NFSServer";
	SupportedProperties = {2, 4, 5, 7, 8};
	Caption = "EFSC:NFSServer";
	FileSharingProtoc

In [102]:
for e in c.EnumerateInstances('CIM_StoragePool'):
    print(e.tomof())

for e in c.EnumerateInstances('Celerra_LocalFileSystem'):
    print(e.tomof())
    
for e in c.EnumerateInstances('Celerra_NFSShare'):
    print(e.tomof())
    
for e in c.EnumerateInstances('CIM_ComputerSystem'):
    print(e.tomof())

instance of Celerra_NonPrimordialStoragePool {
	TotalManagedSpace = 107373133824;
	ElementName = "Pool0";
	InstanceID = "EMC-NSG:root/emc/celerra:APM001432104072007:NPSP:59";
	RemainingManagedSpace = 92340748288;
	HealthState = 5;
	Caption = "NPSP:59";
	OperationalStatus = {2};
	Primordial = False;
	PoolID = "59";
	Usage = 2;
};

instance of Celerra_PrimordialStoragePool {
	TotalManagedSpace = 107373133824;
	ElementName = "PSP1";
	InstanceID = "EMC-NSG:root/emc/celerra:APM001432104072007:PSP:PSP1";
	RemainingManagedSpace = 0;
	HealthState = 5;
	Caption = "PSP:PSP1";
	OperationalStatus = {2};
	Primordial = True;
	PoolID = "PSP1";
	Usage = 2;
};

instance of Celerra_UxfsLocalFileSystem {
	MaxFileNameLength = 255;
	ElementName = "fs_dingj2";
	Name = "13";
	CaseSensitive = True;
	BlockSize = 1024;
	FileSystemType = "EMC_UXFS";
	NumberOfFiles = 0;
	CSCreationClassName = "Celerra_CelerraComputerSystem";
	CasePreserved = True;
	OperationalStatus = {2};
	Caption = "UFS:fs_dingj2";
	AvailableSp

In [103]:
##### Create CIFS Filesystem

fs_service = c.EnumerateInstances('Celerra_FileSystemConfigurationService')[0]
print('##### Using FS service:')
print(fs_service.tomof())

cifs_server = c.EnumerateInstances('Celerra_CIFSServer')[0]
print('##### Using CIFS server:')
print(cifs_server.tomof())

cifs_capabilities = c.Associators(cifs_server.path, ResultClass='SNIA_LocallyAccessibleFileSystemCapabilities')[0]
print('##### Using Filesystem capabilities:')
print(cifs_capabilities.tomof())

result = c.InvokeMethod('CreateGoalSettings', cifs_capabilities.path)
local_access_setting = result[1]['SupportedGoalSettings'][0]
print('##### Using Local access settings:')
print(local_access_setting.tomof())

pool = c.EnumerateInstances('Celerra_NonPrimordialStoragePool')[0]
print('##### Using Pool:')
print(pool.tomof())

result = c.InvokeMethod('SNIA_CreateFileSystem', fs_service.path,
                        Sizes=[pywbemReq.Uint64(10737418240)],
                        FileServer=cifs_server.path,
                        ElementName='fs_cifs_liangr',
                        Pools=[pool.path],
                        LocalAccessSetting=local_access_setting
                       )
print('##### Creating filesystem...')
print('##### Result:')
print(result)

##### Using FS service:
instance of Celerra_FileSystemConfigurationService {
	ElementName = "APM001432104072007";
	Name = "EMC-NSG:root/emc/celerra:APM001432104072007:FSCS";
	Caption = "FSCS:APM001432104072007";
	SystemName = "APM001432104072007";
	CreationClassName = "Celerra_FileSystemConfigurationService";
	SystemCreationClassName = "Celerra_CelerraComputerSystem";
};

##### Using CIFS server:
instance of Celerra_CIFSServer {
	Dedicated = {16};
	Name = "APM001432104072007:vdm_dingj2:CIFS_DINGJ2";
	ElementName = "CIFS_DINGJ2";
	NameFormat = "Other";
	Caption = "CIFSS:CIFS_DINGJ2";
	OperationalStatus = {2};
	CreationClassName = "Celerra_CIFSServer";
	OtherIdentifyingInfo = {"APM001432104072007:vdm_dingj2:CIFS_DINGJ2"};
};

##### Using Filesystem capabilities:
instance of Celerra_LocallyAccessibleFileSystemCapabilities {
	InstanceID = "EMC-NSG:root/emc/celerra:APM001432104072007:LAFSC:vdm_dingj2:CIFS_DINGJ"
       "2";
	Caption = "LAFSC:vdm_dingj2:CIFS_DINGJ2";
	ElementName = "vdm_ding

In [105]:
##### Create CIFS Share #####

cifs_server = c.EnumerateInstances('Celerra_CIFSServer')[0]
print('##### Using CIFS server:')
print(cifs_server.tomof())

cifs_share_service = c.Associators(cifs_server.path, ResultClass= 'CIM_FileExportService')[0]
print('##### Using CIFS share service:')
print(cifs_share_service.tomof())

cifs_share_capabilities = c.Associators(cifs_share_service.path,
                                        AssocClass='CIM_ElementCapabilities',
                                        ResultClass='CIM_ExportedFileShareCapabilities')[0]
print('##### Using CIFS expport capabilities:')
print(cifs_share_capabilities.tomof())

default_no_access, default_ro, default_rw = pywbemReq.Uint16(2), pywbemReq.Uint16(3), pywbemReq.Uint16(4)

cifs_share_name = 'cifs_liangr'
fs_name = 'fs_cifs_liangr'

fs = filter(lambda x: x['ElementName'] == fs_name, c.EnumerateInstances('CIM_FileSystem'))[0]
print('##### Using File System:')
print(fs.tomof())

result = c.InvokeMethod('CreateGoalSettings', cifs_share_capabilities.path)
cifs_goalsettings = result[1]['SupportedGoalSettings'][0]
cifs_goalsettings['DefaultReadWrite'] = default_rw
print('##### Using CIFS Goal Setting:')
print(cifs_goalsettings.tomof())


result = c.InvokeMethod('SNIA_CreateExportedShare',
                        cifs_share_service.path,
                        ElementName=cifs_share_name,
                        Root=fs.path,
                        #SharedElementPath='/',
                        Goal=cifs_goalsettings)
print('##### Creating CIFS share ...')
print('##### Result:')
print(result)
cifs_share = c.GetInstance(result[1]['TheShare'])
print('##### CIFS share created:')
print(cifs_share.tomof())

##### Using CIFS server:
instance of Celerra_CIFSServer {
	Dedicated = {16};
	Name = "APM001432104072007:vdm_dingj2:CIFS_DINGJ2";
	ElementName = "CIFS_DINGJ2";
	NameFormat = "Other";
	Caption = "CIFSS:CIFS_DINGJ2";
	OperationalStatus = {2};
	CreationClassName = "Celerra_CIFSServer";
	OtherIdentifyingInfo = {"APM001432104072007:vdm_dingj2:CIFS_DINGJ2"};
};

##### Using CIFS share service:
instance of Celerra_FileExportService {
	ElementName = "vdm_dingj2:CIFS_DINGJ2";
	Name = "vdm_dingj2:CIFS_DINGJ2";
	Caption = "FES:vdm_dingj2:CIFS_DINGJ2";
	SystemName = "CIFS_DINGJ2";
	CreationClassName = "Celerra_FileExportService";
	SystemCreationClassName = "Celerra_CIFSServer";
};

##### Using CIFS expport capabilities:
instance of Celerra_ExportedFileShareCapabilities {
	CASupported = True;
	ElementName = "CIFS_DINGJ2";
	InstanceID = "EMC-NSG:root/emc/celerra:vdm_dingj2:EFSC:CIFS_DINGJ2";
	SupportedProperties = {2, 4, 5, 7, 8};
	Caption = "EFSC:CIFS_DINGJ2";
	FileSharingProtocol = 3;
};

##### Us

ConnectionError: ('Connection aborted.', BadStatusLine("''",))